In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [17]:
!pip install termcolor

#  ***Importing libaries***

In [18]:
# importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from termcolor import colored, cprint


import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,recall_score,confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier

# ***Importing Datasets***

In [20]:
train = pd.read_csv(r'../input/hp-round-2/Phising_Training Dataset.csv')
test = pd.read_csv(r"../input/hp-round-2/Phising_Testing_Dataset.csv")

In [21]:
train.head()

# ***Data Preprocessing***

In [22]:
for i in range(train.shape[0]):
    if train['Result'][i] == -1:
        train['Result'][i] = 0
        

In [23]:
train.head()

In [24]:
test.drop("key",axis=1,inplace = True)

In [25]:
test.head()

# ***Data Visualization***

In [26]:
import matplotlib
plt.figure(figsize=(22,22))
sns.heatmap(data=train.corr(),annot=True)
plt.title('Heatmap of correlation')
plt.show()

In [27]:
plt.figure(figsize=(8,6))
ax = sns.countplot(data=train,x='SSLfinal_State',hue='Result')
for i in range(len(ax.containers)):
    ax.bar_label(ax.containers[i])
plt.title('SSLfinal_state Vs Result')
plt.show()

In [28]:
plt.figure(figsize=(8,6))
ax = sns.countplot(data=train,x='URL_of_Anchor',hue='Result')
for i in range(len(ax.containers)):
    ax.bar_label(ax.containers[i])
plt.title('URL_of_Anchor Vs Result')
plt.show()

## ***Define Target Variable (y) and Feature Variables (X)***

In [29]:
X = train.iloc[:, :-1]
y = train.iloc[:, -1]

# ***Train Test Split***

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y, random_state=2084)

# ***Modelling***

## **Decision Tree Classifier**

In [31]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf.fit(X,y)
y_pred_dtc = clf.predict(X_test)
print(f'Classification Report of Decision Tree Classifier: \n\n{classification_report(y_test,y_pred_dtc)}')
accuracy = accuracy_score(y_test,y_pred_dtc)
print("Accuracy of the DTC:",accuracy)

## **Random Forest Classifier**

In [32]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators= 200)
rfc.fit(X,y)
y_pred_rfc = rfc.predict(X_test)
print(f'Classification Report of RFC Classifier: \n\n{classification_report(y_test,y_pred_rfc)}')
accuracy = accuracy_score(y_test,y_pred_rfc)
print("Accuracy of the RFC:",accuracy)

## **XGBoost Classifier**

In [34]:
from xgboost import XGBClassifier

from warnings import simplefilter

simplefilter("ignore")
# XGB amaxClassifier Model
xgbc = XGBClassifier(n_estimators= 300,random_state=1)
# Fitting the model
xgbc.fit(X,y)
#predictions on test data
y_pred = xgbc.predict(X_test)
print(f'Classification Report of XGB Classifier: \n\n{classification_report(y_test,y_pred)}')
accuracy = accuracy_score(y_test,y_pred)
print("Accuracy of the XGB:",accuracy)


## **LGBM Classifier**

In [35]:
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier()
lgbm.fit(X,y)
y_pred_lgbm = lgbm.predict(X_test)
print(f'Classification Report of LGBM Classifier: \n\n{classification_report(y_test,y_pred)}')
accuracy = accuracy_score(y_test,y_pred)
print("Accuracy of the LGBM:",accuracy)

## **Stacking Classifier**

In [36]:
from mlxtend.classifier import StackingClassifier
clf_stack = StackingClassifier(classifiers =[lgbm, clf,xgbc,rfc], meta_classifier = rfc,
                               use_probas = True, use_features_in_secondary = True)
model_stack = clf_stack.fit(X, y)   # training of stacked model
pred_stack = model_stack.predict(X_test)
acc_stack = accuracy_score(y_test, pred_stack)

clf_report = classification_report(y_test, pred_stack)

print(f"F1 Score of Stacking Classifier is : {acc_stack}")

print(f"Classification Report : \n{clf_report}")

## **Voting Classifier**

In [37]:
from sklearn.ensemble import VotingClassifier
eclf1 = VotingClassifier(estimators=[('RFC', rfc), ('LGBC', lgbm), ('XGB', xgbc), ('DTC',clf)], voting='soft')
eclf1.fit(X, y)
predictions = eclf1.predict(X_test)
print(classification_report(y_test, predictions))
f1_gb = accuracy_score(y_test, predictions)
print(f"F1 Score of Voting Classifier is : {f1_gb}")

In [38]:
ss = pd.read_csv(r'../input/hp-round-2/sample_submission (1).csv')

In [39]:
ss.head()

In [40]:
preds = clf_stack.predict(test)
preds

In [41]:
for i in range(len(preds)):
    if preds[i] == 0:
        preds[i] = -1

In [42]:
preds

In [43]:
test2 = pd.read_csv(r"../input/hp-round-2/Phising_Testing_Dataset.csv")
test2.head()

In [44]:
ss['key'] = test2['key']
ss['Result'] = preds

In [45]:
ss

In [46]:
ss.to_csv('stack_submission.csv',index=False)